<a href="https://colab.research.google.com/github/dhavalsimaria/MachineLearning/blob/master/NITW/DL%20%26%20AI/DL_%26_AI_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the required libraries

In [0]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Input, Conv2D, Flatten, BatchNormalization, Reshape, Activation, UpSampling2D
from keras.models import Model,Sequential
from keras.datasets import fashion_mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam

In [0]:
# Constants
img_rows, img_cols = 28, 28
opt = adam(lr=1e-3)
dopt = adam(lr=1e-4)
nch = 200
dropout_rate = 0.25

Load the data from dataset

In [0]:
(x_train,y_train), (x_test, y_test) = fashion_mnist.load_data()

Normalize the data

In [0]:
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)

In [0]:
x_train_normalized = (x_train.astype(np.float32))/255

In [51]:
x_train_normalized.shape

(60000, 1, 28, 28)

Define Generator model

In [52]:
g_input = Input(shape=[100]) 
H = Dense(nch*14*14, init='glorot_normal')(g_input) 
H = BatchNormalization()(H) 
H = Activation('relu')(H) 
H = Reshape( [nch, 14, 14] )(H) 
H = UpSampling2D(size=(2, 2))(H)
H = Conv2D(100, (3, 3), activation='relu', padding='same')(H)
H = BatchNormalization()(H)
H = Conv2D(50, (3, 3), activation='relu', padding='same')(H)
H = BatchNormalization()(H)
g_V = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(H)
generator = Model(g_input,g_V)
generator.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
generator.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 39200)             3959200   
_________________________________________________________________
batch_normalization_44 (Batc (None, 39200)             156800    
_________________________________________________________________
activation_15 (Activation)   (None, 39200)             0         
_________________________________________________________________
reshape_14 (Reshape)         (None, 200, 14, 14)       0         
_________________________________________________________________
up_sampling2d_14 (UpSampling (None, 400, 28, 14)       0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 400, 28, 100)      127

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(39200, kernel_initializer="glorot_normal")`
  


Define Discriminator Model

In [59]:
shp = x_train.shape[1:]
d_input = Input(shape=shp)
H = Conv2D(256, (5, 5), subsample=(2, 2), border_mode = 'same', activation='relu')(d_input)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
H = Conv2D(512, (5, 5), subsample=(2, 2), border_mode = 'same', activation='relu')(H)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
H = Flatten()(H)
H = Dense(256)(H)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
d_V = Dense(2,activation='softmax')(H)
discriminator = Model(d_input,d_V)
discriminator.compile(loss='categorical_crossentropy', optimizer=dopt)
discriminator.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 1, 28, 28)         0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 1, 14, 256)        179456    
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 1, 14, 256)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 14, 256)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 1, 7, 512)         3277312   
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 1, 7, 512)         0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 1, 7, 512)         0  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="relu", strides=(2, 2), padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (5, 5), activation="relu", strides=(2, 2), padding="same")`
  
